<img width=50 src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Sample notebook to explore biomass dataset

Authors: Oriana Chegwidden and Cindy Chiao

To run this notebook locally and produce the loveliest-possible figures, you'll
want to have the CarbonPlan styles installed.


In [ ]:
import xarray as xr
import pandas as pd
from carbonplan_styles.colors import colors
from carbonplan_styles.mpl import set_theme
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

theme = "dark"
set_theme(style=f"carbonplan_{theme}")
c = colors(theme)

## Load in global 1 degree raster


In [ ]:
ds = xr.open_zarr(
    "s3://carbonplan-scratch/junk/v0_one_degree_raster.zarr", consolidated=True
)
da = (ds.rename({"emissions": "Emissions [Mt CO2]"}) / 1e6)[
    "Emissions [Mt CO2]"
]

## Global map of cumulative emisisons over 2001-2020


In [ ]:
plt.figure(figsize=(12, 10))
s = da.where(da > 0.0).sum("year")
s = s.where(s > 0)
p = s.plot(
    cmap=f"fire_{theme}",
    subplot_kws=dict(projection=ccrs.Robinson()),
    transform=ccrs.PlateCarree(),
    robust=True,
    cbar_kwargs={"orientation": "horizontal", "pad": 0.02, "shrink": 0.55},
)
p.axes.set_global()
p.axes.coastlines(color=c["secondary"])
p.axes.set_title("Cumulative emissions from forests (2001-2020)")

## Global emisisons for every year


In [ ]:
s = da.where(da > 0.0)
s = s.where(s > 0)
fg = da.plot(
    col="year",
    col_wrap=5,
    vmax=1,
    cmap=f"fire_{theme}",
    subplot_kws=dict(projection=ccrs.Robinson()),
    transform=ccrs.PlateCarree(),
    robust=True,
    cbar_kwargs={"orientation": "horizontal", "pad": 0.02, "shrink": 0.55},
)
for panel, year in zip(fg.axes.flatten(), da.year.values):
    panel.set_title(year)
    panel.set_global()
    panel.coastlines(color=c["secondary"])

## Timeseries of global forest emissions over time


In [ ]:
da = da.assign_coords({"year": pd.date_range("2001", "2020", freq="YS")})
da.sum(dim=["x", "y"]).plot(color=c["orange"])
plt.xlabel("")
plt.title("Annual global emissions from forests")

## Load in a sample tile from the underlying 30 m dataset


In [ ]:
path_template = (
    "s3://carbonplan-climatetrace/{version}/tiles/30m/{tile}_tot.zarr/"
)

## Example of forest losses in the Amazon


In [ ]:
tile = "00N_060W"
version = "v0.4"

ds = xr.open_zarr(
    path_template.format(tile=tile, version=version), consolidated=False
)
ds = ds.rename({"emissions": "Emissions [t CO2]"})
sub = ds.sel(lat=slice(-6.225, -6.35), lon=slice(-53.45, -53.35))

In [ ]:
s = sub.where(sub > 0.0)
s["Emissions [t CO2]"].plot(
    col="year",
    col_wrap=5,
    cmap=f"fire_{theme}",
    subplot_kws=dict(projection=ccrs.Robinson()),
    transform=ccrs.PlateCarree(),
    robust=True,
    cbar_kwargs={"orientation": "horizontal", "pad": 0.02, "shrink": 0.55},
)